# 简单策略回测

1. 使用yfinance获取股票数据
创建数据表，选择股票代码601318，其中SS代表上海证券交易所

字段说明：
Open:开盘价; High:最高价; Low:最低价; Close:收盘价; 
Adj Close:调整后的收盘价; Volume:成交量

2. 简单策略
当股价下跌，在下一个交易日前挂单买入
反之，当股价上涨，则在下一个交易日前卖出
为了体现股票的真实价值, 选取股票调整后的收盘价Adj Close作为股票的真实价格。同时为了便于计算下单数量, 用0替换控制NAN

In [1]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

start_date = "2020-01-01"
end_date = "2020-03-18"
stock_code = "601318.ss"    # 中国平安，ss代表上海证券交易所

data = yf.download(stock_code, start=start_date, end=end_date)
data.head()

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-01-02,85.900002,86.790001,85.879997,86.120003,72.726852,77825207
2020-01-03,86.809998,86.879997,85.900002,86.199997,72.794411,59498001
2020-01-06,85.919998,86.870003,85.500000,85.599998,72.287727,63644804
2020-01-07,86.010002,86.459999,85.669998,86.150002,72.752190,45218832
2020-01-08,85.000000,85.000000,85.000000,85.000000,71.781029,62805311


In [ ]:
# 简单策略

signal = pd.DataFrame(index=data.index)     # 不影响原始数据，创建新表
signal['price'] = data['Adj Close']         # 使用调整后的收盘价(Adj close)作为价格
signal['diff'] = signal['price'].diff()     # 储存股价的变化,按照当日的收盘价减去前一个交易日的收盘价来计算
signal = signal.fillna(0.0)                 # 用0来填补NAN
signal['signal'] = np.where(signal['diff']>=0, 0, 1)    # 股价上涨则标记为1，下跌则标记为0
signal['order'] = signal['signal'].diff()*100           # 订单，买入或卖出100股，即1手

# 开始回测
initial_cash = 20000.00                     # 考虑初始股价，初始金额为2万人民币